## Generator là gì?

Như đã nói qua ở phần trước, generator đơn giản là một function trả về một channel, mục tiêu là để tách biệt:
- Values, data được tạo như thế nào
- Và các giá trị này được consume ở đâu

Trong function, goroutine sẽ tạo data và đẩy vào channel.  
Consumer chỗ khác chỉ đơn giản là tiêu thụ data ở đây mà không cần quan tâm nó được tạo như thế nào.  
Template chung là như ở dưới

In [ ]:
// Generator function signature
func generator() <-chan Type {
    ch := make(chan Type)
    go func() {
        // Produce values
        // Send to ch
    }()
    return ch
}

#### Ví dụ:

In [3]:
// Generator: trả về một channel mà tạo ra các số từ 1 tới n
func count(n int) <-chan int {
    out := make(chan int)

    go func() {
        for i := 1; i <= n; i++ {
            out <- i
        }
        close(out)
    }()

    return out
}

func main() {
    nums := count(5)

    // dùng for...range để loop channel
    for v := range nums {
        fmt.Println(v)
    }
}

1
2
3
4
5


## Rob Pike's Classic Example

Classic generator của Rob Pike, tương tự ví dụ ở trên, điểm khác:
- Ví dụ ở trên có close để đóng channel
- Ví dụ của Rob Pike ko có ý định đóng channel khi xong, mà dùng time.Sleep cho chạy hoài tới khi main kết thúc
- Đóng một channel chỉ là optional

In [4]:
func boring(msg string) <-chan string {
    c := make(chan string)
    go func() {
        for i := 0; ; i++ {
            c <- fmt.Sprintf("%s %d", msg, i)
            time.Sleep(time.Duration(rand.Intn(1e3)) * time.Millisecond)
        }
    }()
    return c
}

func main() {
    joe := boring("Joe")
    ann := boring("Ann")
    
    for i := 0; i < 10; i++ {
        fmt.Println(<-joe)
        fmt.Println(<-ann)
    }
    fmt.Println("You're both boring; I'm leaving.")
}

Joe 0
Ann 0
Joe 1
Ann 1
Joe 2
Ann 2
Joe 3
Ann 3
Joe 4
Ann 4
Joe 5
Ann 5
Joe 6
Ann 6
Joe 7
Ann 7
Joe 8
Ann 8
Joe 9
Ann 9
You're both boring; I'm leaving.


## Tại sao dùng Generator?

### 1. Encapsulation - tính đóng gói
Caller/Consumer không cần quan tâm tới quản lý goroutine thế nào

In [ ]:
// Without generator - manual goroutine management
ch := make(chan int)
go func() {
    for i := 0; i < 10; i++ {
        ch <- i
    }
    close(ch)
}()

// With generator - clean interface
numbers := generateNumbers(10)

### 2. Composability
Các generators có thể chained với nhau

In [ ]:
func double(in <-chan int) <-chan int {
    out := make(chan int)
    go func() {
        for n := range in {
            out <- n * 2
        }
        close(out)
    }()
    return out
}

// Compose generators
numbers := generateNumbers(10)
doubled := double(numbers)

### 3. Separation of Concerns
Phân chia logic giữa producer và consumer

## Common Generator Patterns

### 1. Finite Generator
Này giống ví dụ đầu tiên, tạo ra một hữu hạn dãy số

In [8]:
func counter(max int) <-chan int {
    ch := make(chan int)
    go func() {
        for i := 0; i < max; i++ {
            ch <- i
        }
        close(ch)
    }()
    return ch
}

func main() {
    for n := range counter(5) {
        fmt.Println(n) // 0, 1, 2, 3, 4
    }
}

0
1
2
3
4


### 2. Infinite Generator
Tạo ra vô hạn số, ví dụ của Rob Pike ở trên

In [10]:
func fibonacci() <-chan int {
    ch := make(chan int)
    go func() {
        a, b := 0, 1
        for {
            ch <- a
            a, b = b, a+b
        }
    }()
    return ch
}

func main() {
    fib := fibonacci()
    for i := 0; i < 10; i++ {
        fmt.Println(<-fib)
    }
}

0
1
1
2
3
5
8
13
21
34


### 3. Generator with State

In [12]:
func randomGenerator(seed int64) <-chan int {
    ch := make(chan int)
    go func() {
        r := rand.New(rand.NewSource(seed))
        for {
            ch <- r.Intn(100)
        }
    }()
    return ch
}

Đây là ví dụ về tạo ra số ngẫu nhiên, nhưng giải thuật để tạo ra số ngẫu nhiên cần phải giữ state mỗi lần tạo ra số mới, nghĩa là mỗi lần gọi tạo ra 1 số random thì state của hàm này là không đổi

### 4. Generator with Cleanup

In [13]:
func generate(done <-chan bool) <-chan int {
    ch := make(chan int)
    go func() {
        defer close(ch)
        for i := 0; ; i++ {
            select {
            case ch <- i:
                // Successfully sent
            case <-done:
                return
            }
        }
    }()
    return ch
}

func main() {
    done := make(chan bool)
    numbers := generate(done)
    
    // Consume some values
    for i := 0; i < 5; i++ {
        fmt.Println(<-numbers)
    }
    
    // Stop the generator
    close(done)
}

0
1
2
3
4


Đây là một ví dụ kinh điển về channel done có kiểu boolean để kết thúc một func. Nhưng kiểu boolean vẫn tốn 1 bit, có trick sử dụng empty struct cho việc này:

In [ ]:
func generate(done <-chan struct{}) <-chan int {
    ch := make(chan int)
    go func() {
        defer close(ch)
        for i := 0; ; i++ {
            select {
            case ch <- i:
            case <-done:
                return
            }
        }
    }()
    return ch
}

func main() {
    done := make(chan struct{}) 
    numbers := generate(done)

    // Consume some values
    for i := 0; i < 5; i++ {
        fmt.Println(<-numbers)
    }

    close(done)
}

### `close(done)` vs `done <- struct{}{}`

In [ ]:
go func() {
    <-done // will unblock as soon as channel is closed
    fmt.Println("Stopped")
}()

close(done) // all listeners stop

In [ ]:
go func() {
    <-done
    fmt.Println("Got the signal")
}()

done <- struct{}{} // only one goroutine receives this

- `close(done)`: báo toàn bộ goroutine stop (đây cũng là cách `context.Context` hoạt động
- `done <- struct{}{}`: để đánh thức 1 goroutine, có thể gửi nhiều signal như job queue   
Xem ví dụ bên dưới để thấy rõ hơn

In [14]:
func worker(id int, done <-chan struct{}) {
    for {
        select {
        case <-done:
            fmt.Printf("Worker %d stopping\n", id)
            return
        default:
            fmt.Printf("Worker %d working...\n", id)
            time.Sleep(300 * time.Millisecond)
        }
    }
}

func main() {
    done := make(chan struct{})

    for i := 1; i <= 3; i++ {
        go worker(i, done)
    }

    time.Sleep(time.Second)
    fmt.Println(">>> Closing done channel (broadcast)")
    close(done) // all workers get the signal

    time.Sleep(time.Second)
}

Worker 3 working...
Worker 1 working...
Worker 2 working...
Worker 2 working...
Worker 3 working...
Worker 1 working...
Worker 1 working...
Worker 2 working...
Worker 3 working...
Worker 3 working...
Worker 2 working...
Worker 1 working...
>>> Closing done channel (broadcast)
Worker 1 stopping
Worker 3 stopping
Worker 2 stopping


In [15]:
package main

import (
    "fmt"
    "time"
)

func worker(id int, done <-chan struct{}) {
    for {
        select {
        case <-done:
            fmt.Printf("Worker %d stopping\n", id)
            return
        default:
            fmt.Printf("Worker %d working...\n", id)
            time.Sleep(300 * time.Millisecond)
        }
    }
}

func main() {
    done := make(chan struct{})

    for i := 1; i <= 3; i++ {
        go worker(i, done)
    }

    time.Sleep(time.Second)
    fmt.Println(">>> Sending one struct{}{} (single signal)")
    done <- struct{}{} // only ONE worker gets this

    time.Sleep(time.Second)
}

Worker 3 working...
Worker 1 working...
Worker 2 working...
Worker 2 working...
Worker 3 working...
Worker 1 working...
Worker 3 working...
Worker 1 working...
Worker 2 working...
Worker 2 working...
Worker 1 working...
Worker 3 working...
>>> Sending one struct{}{} (single signal)
Worker 2 stopping
Worker 3 working...
Worker 1 working...
Worker 1 working...
Worker 3 working...
Worker 3 working...
Worker 1 working...
Worker 1 working...
Worker 3 working...


## Advanced Generator Techniques (pending...)

## Best Practices

### 1. Return Read-Only Channels

In [ ]:
// ✅ Prevents accidental sends by consumer
func generator() <-chan int {
    // ...
}

### 2. Close Channels When Done

In [ ]:
// ✅ Allows range loops
func finite() <-chan int {
    ch := make(chan int)
    go func() {
        defer close(ch)
        for i := 0; i < 10; i++ {
            ch <- i
        }
    }()
    return ch
}

### 3. Handle Cancellation

In [ ]:
// ✅ Graceful shutdown
func cancellable(ctx context.Context) <-chan int {
    ch := make(chan int)
    go func() {
        defer close(ch)
        for i := 0; ; i++ {
            select {
            case ch <- i:
            case <-ctx.Done():
                return
            }
        }
    }()
    return ch
}

### 4. Document Generator Behavior

In [ ]:
// generatePrimes returns a channel that produces prime numbers
// until the context is cancelled. The channel is closed when done.
func generatePrimes(ctx context.Context) <-chan int

## Common Pitfalls

## Performance Tips

#### From Rob Pike's guidance and Go best practices:
1. Small buffers (1-10) often sufficient
2. Benchmark buffer sizes for your use case
3. Consider batching for high-frequency generators
4. Use sync.Pool for temporary objects in generators

## Real-World Examples